In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
master_dataset_final = pd.read_csv('master_dataset_final.csv')

freq_vect = CountVectorizer(analyzer='word', ngram_range=(1,2),max_features=6892)
count_matrix = freq_vect.fit_transform(master_dataset_final.soup)
# df = pd.DataFrame(data=count_array, columns = count_vect.get_feature_names_out())
# print(count_matrix)


cosine_sim = cosine_similarity(count_matrix)
cosine_df = pd.DataFrame(cosine_sim, index=master_dataset_final['title'], columns=master_dataset_final['title'])
cosine_df.to_csv("cosine_similarity_matrix.csv")

# Step 5: Define Recommendation Function
def recommend_movies(movie_name, top_n=5):
    if movie_name not in cosine_df.index:
        return f"'{movie_name}' not found in movie list."

    movie_idx = cosine_df.index.get_loc(movie_name)
    sim_scores = list(enumerate(cosine_sim[movie_idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]

    recommendations = []
    for idx, score in sim_scores:
        recommendations.append({
            'title': master_dataset_final.iloc[idx]['title'],
            'main_director': master_dataset_final.iloc[idx]['main_director'],
            'release_date': master_dataset_final.iloc[idx]['release_date'],
            'similarity_score': round(score, 3)
        })

    return pd.DataFrame(recommendations)

# Example Usage
recommendations = recommend_movies("Avatar", top_n=8)
print(recommendations)

                                         title  main_director release_date  \
0                                       Aliens  James Cameron   1986-07-18   
1                                    True Lies  James Cameron   1994-07-14   
2                                      Titanic  James Cameron   1997-11-18   
3                               The Terminator  James Cameron   1984-10-26   
4                   Terminator 2: Judgment Day  James Cameron   1991-07-01   
5                      Star Trek Into Darkness    J.J. Abrams   2013-05-05   
6                                        Alien   Ridley Scott   1979-05-25   
7  Valerian and the City of a Thousand Planets     Luc Besson   2017-07-20   

   similarity_score  
0             0.548  
1             0.446  
2             0.427  
3             0.397  
4             0.390  
5             0.183  
6             0.120  
7             0.120  
